In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from multiprocessing import Pool
import multiprocessing

In [3]:
def LJpot(r):
    #r = np.sqrt(np.sum((X1-X2)**2))
    return 4*(1/r**12-1/r**6)

def LJF(r):
    #r = np.sqrt(np.sum((X1-X2)**2))
    if(0<r):
        return -24./r**2*((r/2.)**-12 - r**-6)
    else:
        return 0
    
def LJpot2(r):
    #r = np.sqrt(np.sum((X1-X2)**2))
    return 4*(1/r**6-1/r**3)

def LJF2(r):
    #r = np.sqrt(np.sum((X1-X2)**2))
    return -24./r*(r**-6*2**12 - r**-3)

In [4]:
N_len = 4
steps = 10*500

N = 4*N_len**3

dt = 0.005
t = dt*steps

r = np.zeros((int(t/dt),N,3))
p = np.zeros((int(t/dt),N,3))
U = np.zeros(int(t/dt)-1)
K = np.zeros(int(t/dt)-1)
E = np.zeros(int(t/dt)-1)
P = np.zeros(int(t/dt)-1)
T = np.zeros(int(t/dt)-1)

a = (4/0.55)**(1./3.)

rc = N_len*a/(6*a)*5

V = (6*a)**3

In [5]:
for i in range(N_len):
    for j in range(N_len):
        for k in range(N_len):
            r[0][N_len**3*0+N_len**2*i+N_len*j+k][0],r[0][N_len**3*0+N_len**2*i+N_len*j+k][1],r[0][N_len**3*0+N_len**2*i+N_len*j+k][2] = k,j,i
            r[0][N_len**3*1+N_len**2*i+N_len*j+k][0],r[0][N_len**3*1+N_len**2*i+N_len*j+k][1],r[0][N_len**3*1+N_len**2*i+N_len*j+k][2] = k+1./2,j+1./2,i
            r[0][N_len**3*2+N_len**2*i+N_len*j+k][0],r[0][N_len**3*2+N_len**2*i+N_len*j+k][1],r[0][N_len**3*2+N_len**2*i+N_len*j+k][2] = k,j+1./2,i+1./2
            r[0][N_len**3*3+N_len**2*i+N_len*j+k][0],r[0][N_len**3*3+N_len**2*i+N_len*j+k][1],r[0][N_len**3*3+N_len**2*i+N_len*j+k][2] = k+1./2,j,i+1./2

r = r*a

for i in range(N_len):
    for j in range(N_len):
        for k in range(N_len):
            p[0][N_len**3*0+N_len**2*i+N_len*j+k][0],p[0][N_len**3*0+N_len**2*i+N_len*j+k][1],p[0][N_len**3*0+N_len**2*i+N_len*j+k][2] = np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5)
            p[0][N_len**3*1+N_len**2*i+N_len*j+k][0],p[0][N_len**3*1+N_len**2*i+N_len*j+k][1],p[0][N_len**3*1+N_len**2*i+N_len*j+k][2] = np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5)
            p[0][N_len**3*2+N_len**2*i+N_len*j+k][0],p[0][N_len**3*2+N_len**2*i+N_len*j+k][1],p[0][N_len**3*2+N_len**2*i+N_len*j+k][2] = np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5)
            p[0][N_len**3*3+N_len**2*i+N_len*j+k][0],p[0][N_len**3*3+N_len**2*i+N_len*j+k][1],p[0][N_len**3*3+N_len**2*i+N_len*j+k][2] = np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5),np.random.normal(loc=0,scale=0.2**0.5)
        
R = np.zeros((27,3))

rvec = np.array([0,0,0])

R[0] = rvec + np.array([1, 1, 1])*N_len*a
R[1] = rvec + np.array([1, 1, 0])*N_len*a
R[2] = rvec + np.array([1, 1, -1])*N_len*a

R[3] = rvec + np.array([1, 0, 1])*N_len*a
R[4] = rvec + np.array([1, 0, 0])*N_len*a
R[5] = rvec + np.array([1, 0, -1])*N_len*a

R[6] = rvec + np.array([1, -1, 1])*N_len*a
R[7] = rvec + np.array([1, -1, 0])*N_len*a
R[8] = rvec + np.array([1, -1, -1])*N_len*a

R[9] = rvec + np.array([0, 1, 1])*N_len*a
R[10] = rvec + np.array([0, 1, 0])*N_len*a
R[11] = rvec + np.array([0, 1, -1])*N_len*a

R[12] = rvec + np.array([0, 0, 1])*N_len*a
R[13] = rvec + np.array([0, 0, 0])*N_len*a
R[14] = rvec + np.array([0, 0, -1])*N_len*a

R[15] = rvec + np.array([0, -1, 1])*N_len*a
R[16] = rvec + np.array([0, -1, 0])*N_len*a
R[17] = rvec + np.array([0, -1, -1])*N_len*a

R[18] = rvec + np.array([-1, 1, 1])*N_len*a
R[19] = rvec + np.array([-1, 1, 0])*N_len*a
R[20] = rvec + np.array([-1, 1, -1])*N_len*a

R[21] = rvec + np.array([-1, 0, 1])*N_len*a
R[22] = rvec + np.array([-1, 0, 0])*N_len*a
R[23] = rvec + np.array([-1, 0, -1])*N_len*a

R[24] = rvec + np.array([-1, -1, 1])*N_len*a
R[25] = rvec + np.array([-1, -1, 0])*N_len*a
R[26] = rvec + np.array([-1, -1, -1])*N_len*a


In [6]:
for i in range(N):
    rvec = (r[0]-r[0][i])
    for j in range(N):
        R2 = R + rvec[j]
        d = np.sum(R2**2,axis=1)        
        arg_min = np.argmin(d)
        rmag = d[arg_min]
        if(0<rmag<rc**2):
            U[0] += 4*(1/rmag**6-1/rmag**3)
            
U[0] = U[0]/2/N
K[0] = 1./2*np.sum(p[0]**2)/N
T[0] = 2/3*K[0]
P[0] += N/V*T[0]

for i in range(N):
    r[1][i] = r[0][i] + p[0][i]*dt

r = r%(N_len*a)

In [7]:
𝑐def evolve(it):
    U2 = 0
    P2 = 0
    i2,t2 = it
    temp2 = np.zeros(3)
    rvec2 = (r[t2]-r[t2][i2])
    for j in range(N):
        R22 = R + rvec2[j]
        d2 = np.sum(R22**2,axis=1) 
        arg_min2 = np.argmin(d2)
        rmag2 = d2[arg_min2]
        if(0<rmag2<rc**2):
                temp2 += (-24.*(rmag2**-7*2 - rmag2**-4))*R22[arg_min2]
                U2 += 4*(1/rmag2**6-1/rmag2**3)

    del_r = r[t2][i2]-r[t2-1][i2]
    for j in range(3):
        if(del_r[j]**2>rc**2):
            del_r[j] = np.sign(del_r[j])*(np.abs(del_r[j])-N_len*a)

    return [r[t2][i2] + del_r + temp2*dt**2,U2/2/N,P2/2/N,(del_r+1./2*temp2*dt**2)/dt]

In [8]:
t1 = time.time()

iter_i = []
for i in range(N):
    iter_i.append((i,1))

for ti in range(1,int(t/dt)-1):
    
    pool = Pool(processes=7)
    res = pool.map(evolve,iter_i,chunksize=int(N/6.5))
    pool.close()
    
    for i in range(N):
        r[ti+1][i] = res[i][0]
        p[ti][i] = res[i][-1]
        iter_i[i] = (i,ti+1)
    
    r = r%(N_len*a)
    
    U[ti] = np.sum(np.array(res)[:,1]) 
    K[ti] = 1./2*np.sum(p[ti]**2)/N
    T[ti] = 2/3*K[ti]
    P[ti] = res[i][2]+N/V*T[ti]

print(time.time()-t1)

Process ForkPoolWorker-3853:
Process ForkPoolWorker-3854:
Process ForkPoolWorker-3857:
Process ForkPoolWorker-3856:
Process ForkPoolWorker-3855:
Process ForkPoolWorker-3852:
Process ForkPoolWorker-3851:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/pranav/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/pranav/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/pranav/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranav/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranav/miniconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/pranav/miniconda3/lib/python3.7/multiproces

In [ ]:
plt.plot(range(0,int(t/dt)-1),T)
plt.plot(range(0,int(t/dt)-1),U)
plt.plot(range(0,int(t/dt)-1),K)
plt.plot(range(0,int(t/dt)-1),U+K)

In [15]:
import csv

RR = r.tolist()
with open('r.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(RR)


PP = p.tolist()
with open('p.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(PP)
    

np.savetxt('T.csv',T)
np.savetxt('U.csv',U)
np.savetxt('K.csv',K)

In [ ]:
with open('r.csv', 'r') as f:
    reader = csv.reader(f)
    examples = list(reader)

print(examples)
nwexamples = []
for row in examples:
    nwrow = []
    for r in row:
        nwrow.append(eval(r))
    nwexamples.append(nwrow)
print(nwexamples)

In [10]:
print(U[0],K[0],T[0])

-3.6855251885403617 0.3159600140494896 0.21064000936632638
